In [ ]:
import os

In [ ]:
with open('/medical-data/zsxm/e3v/resized_data/labels.txt', 'r') as f:
    lines = f.readlines()

label_dict = {}
for line in lines:
    video, label = line.split()
    video = os.path.basename(video).lower()
    label_dict[video] = label

In [ ]:
root = '/medical-data/zsxm/e3v/resized_data/'

for cate in ['train', 'val', 'test']:
    with open(os.path.join(root, f'{cate}_labels.txt'), 'w') as lf:
        for video in sorted(os.listdir(os.path.join(root, cate))):
            label = label_dict[video.lower()]
            lf.write(f'{os.path.join(root, cate, video)}\t{label}\n')

In [ ]:
from mmcv import load

annotation = load('/medical-data/zsxm/e3v/resized_data/val_annotations_error.pkl')

In [ ]:
print(annotation[0].keys())

In [ ]:
import os
file_dir = set()
for v in os.listdir('/medical-data/zsxm/e3v/resized_data/val/'):
    file_dir.add(os.path.splitext(v.lower())[0])

anno_dir = set()
for it in annotation:
    anno_dir.add(it['frame_dir'].lower())

print(file_dir == anno_dir)

In [ ]:
annotation[0]['keypoint'].shape

In [ ]:
import numpy as np

keypoint = annotation[1]['keypoint']
print(annotation[1]['frame_dir'])

In [ ]:
offset = np.linalg.norm(np.diff(keypoint.astype(np.double), axis=2), ord=2, axis=3).sum(axis=-1).mean(axis=-1)
print(offset)

In [2]:
from pyskl.utils.visualize import Vis3DPose, Vis2DPose
from mmcv import load

annotation = load('examples/lwf_anno.pkl')
anno = annotation[0]
vid = Vis2DPose(anno, thre=0.2, out_shape=(480, 856), layout='coco', fps=15, video=None)
vid.ipython_display(fps=15)

100%|██████████| 62/62 [00:00<00:00, 3163.43it/s]


Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [6]:
import copy
import numpy as np

anno2 = copy.deepcopy(anno)

offset = np.linalg.norm(np.diff(anno2['keypoint'].astype(np.double), axis=1), ord=2, axis=3).sum(axis=-1).mean(axis=-1)

# kp = anno2['keypoint'].astype(np.double)
# gravitycenter = kp.mean(axis=2, keepdims=True)
# offset = np.linalg.norm(np.diff(kp-gravitycenter, axis=1), ord=2, axis=3).sum(axis=-1).mean(axis=-1)

idx = np.argmax(offset)
print(offset, idx)
anno2['num_person_raw'] = 1
anno2['keypoint'] = anno2['keypoint'][idx:idx+1]
anno2['keypoint_score'] = anno2['keypoint_score'][idx:idx+1]
vid = Vis2DPose(anno2, thre=0.2, out_shape=(480, 856), layout='coco', fps=15, video=None)
vid.ipython_display()

[204.30687847 476.4038967 ] 1


100%|██████████| 62/62 [00:00<00:00, 5688.56it/s]


Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [6]:
from mmcv import load, dump
import numpy as np

input_dir = '/medical-data/zsxm/e3v/resized_data/val_annotations_error.pkl'
output_dir = '/medical-data/zsxm/e3v/resized_data/val_annotations_error_one.pkl'

annotation = load(input_dir)
for anno in annotation:
    offset = np.linalg.norm(np.diff(anno['keypoint'].astype(np.double), axis=1), ord=2, axis=3).sum(axis=-1).mean(axis=-1)
    idx = np.argmax(offset)
    print(anno['frame_dir'], anno['num_person_raw'], offset, idx)
    anno['keypoint'] = anno['keypoint'][idx:idx+1]
    anno['keypoint_score'] = anno['keypoint_score'][idx:idx+1]
    anno['num_person_raw'] = 1

dump(annotation, output_dir)


lwf_running_f_back_1_00-00-02 2 [155.43610252  52.58674935] 0
lwf_running_f_back_1_00-00-09 2 [131.73902383  26.07734037] 0
lwf_running_f_back_1_00-00-17 2 [181.80882521  81.68765165] 0
lwf_running_f_back_1_00-00-24 2 [482.0008519  424.71028487] 0
lwf_running_f_back_1_00-00-32 2 [443.15608073 373.88910474] 0
lwf_running_f_back_1_00-00-38 2 [523.7762351 460.839954 ] 0
lwf_running_f_back_1_00-00-44 2 [380.84298018 286.88020033] 0
lwf_running_f_back_1_00-00-51 2 [571.6077047  529.36232088] 0
lwf_running_f_back_1_00-00-57 2 [532.27272847 464.68828251] 0
lwf_running_f_back_1_00-01-04 2 [508.60482003 445.35681097] 0
lwf_running_f_back_1_00-01-11 2 [336.0866297  262.31612205] 0
lwf_running_f_back_1_00-01-17 2 [138.13782818  30.00371392] 0
lwf_running_f_back_1_00-01-25 2 [253.3928652  162.14659222] 0
lwf_running_f_back_1_00-01-32 2 [439.05284689 372.16288774] 0
lwf_running_f_back_1_00-01-39 2 [265.95776348 177.25348613] 0
lwf_running_f_back_1_00-01-46 2 [198.54556934 100.88249908] 0
lwf_runnin

In [7]:
from mmcv import load, dump
import numpy as np

input_dir = '/medical-data/zsxm/e3v/resized_data/val_annotations_error.pkl'
output_dir = '/medical-data/zsxm/e3v/resized_data/val_annotations_error_two.pkl'

annotation = load(input_dir)
for anno in annotation:
    kp = anno['keypoint'].astype(np.double)
    gravitycenter = kp.mean(axis=2, keepdims=True)
    offset = np.linalg.norm(np.diff(kp-gravitycenter, axis=1), ord=2, axis=3).sum(axis=-1).mean(axis=-1)
    idx = np.argmax(offset)
    print(anno['frame_dir'], anno['num_person_raw'], offset, idx)
    anno['keypoint'] = anno['keypoint'][idx:idx+1]
    anno['keypoint_score'] = anno['keypoint_score'][idx:idx+1]
    anno['num_person_raw'] = 1

dump(annotation, output_dir)

lwf_running_f_back_1_00-00-02 2 [92.12675698 34.06674472] 0
lwf_running_f_back_1_00-00-09 2 [86.52356443 25.08678211] 0
lwf_running_f_back_1_00-00-17 2 [101.7982161   45.24952409] 0
lwf_running_f_back_1_00-00-24 2 [201.67303728 166.59169118] 0
lwf_running_f_back_1_00-00-32 2 [189.98376825 149.72361252] 0
lwf_running_f_back_1_00-00-38 2 [223.4472819  182.61548608] 0
lwf_running_f_back_1_00-00-44 2 [175.02859546 119.00710518] 0
lwf_running_f_back_1_00-00-51 2 [233.36145803 209.72075901] 0
lwf_running_f_back_1_00-00-57 2 [227.96596021 187.15874616] 0
lwf_running_f_back_1_00-01-04 2 [216.88330696 176.3698031 ] 0
lwf_running_f_back_1_00-01-11 2 [157.77217168 108.26762722] 0
lwf_running_f_back_1_00-01-17 2 [89.63453211 30.88521981] 0
lwf_running_f_back_1_00-01-25 2 [129.35998774  77.36948273] 0
lwf_running_f_back_1_00-01-32 2 [193.15463571 153.76732962] 0
lwf_running_f_back_1_00-01-39 2 [133.92911746  84.20154945] 0
lwf_running_f_back_1_00-01-46 2 [106.76655251  55.53083283] 0
lwf_running_f_

In [2]:
from mmcv import load, dump
import os.path as osp

root = '/medical-data/zsxm/e3v/resized_data/'

cates = [
    ('train', 'train_annotations_track.pkl'),
    ('valid', 'val_annotations_track.pkl'),
    ('test', 'test_annotations_track.pkl'),
]

annotations = []
split = {}
for c, cf in cates:
    annos = load(osp.join(root, cf))
    annotations.extend(annos)
    split[c] = [x['frame_dir'] for x in annos]

dump(dict(split=split, annotations=annotations), osp.join(root, 'annotations_track.pkl'))

In [10]:
from mmcv import load, dump
import os.path as osp
root = '/medical-data/zsxm/e3v/resized_data/'
dataset = load(osp.join(root, 'annotations_track.pkl'))

In [11]:
dataset['split']['train'] = dataset['split']['train'][:100]
dataset['split']['valid'] = dataset['split']['valid'][:100]
dataset['split']['test'] = dataset['split']['test'][:100]

In [12]:
dump(dataset, osp.join(root, 'annotations_demo.pkl'))

In [8]:
a = load(osp.join(root, 'annotations_demo.pkl'))

In [9]:
a

{'split': {'train': 'hqh_elliptical_m_back_1_00-17-20',
  'valid': 'lwf_running_f_back_1_00-07-19',
  'test': 'xwy_running_m_back_1_00-05-50'},
 'annotations': [{'frame_dir': 'hqh_elliptical_m_back_1_00-00-40',
   'label': 0.3,
   'img_shape': (480, 856),
   'total_frames': 152,
   'num_person_raw': 1,
   'keypoint': array([[[[482.8 , 131.5 ],
            [482.8 , 124.9 ],
            [479.2 , 124.9 ],
            ...,
            [496.  , 324.  ],
            [396.2 , 400.5 ],
            [496.  , 397.2 ]],
   
           [[483.8 , 129.9 ],
            [483.8 , 126.56],
            [483.8 , 123.1 ],
            ...,
            [494.  , 326.2 ],
            [395.8 , 397.5 ],
            [494.  , 400.8 ]],
   
           [[485.  , 130.5 ],
            [485.  , 127.1 ],
            [485.  , 123.75],
            ...,
            [491.8 , 329.8 ],
            [397.2 , 394.  ],
            [495.2 , 407.5 ]],
   
           ...,
   
           [[477.8 , 139.5 ],
            [477.8 , 132.4 ]